# Fully Convolutional Interior/Edge Segmentation for 2D Data

---

Classifies each pixel as either Cell Edge, Cell Interior, or Background.

There are 2 different Cell Edge classes (Cell-Cell Boundary and Cell-Background Boundary)

In [1]:
import os
import errno

import os
import errno
import functools

import numpy as np

import tensorflow as tf
import deepcell.utils


import numpy as np

import deepcell

### Load the training data

In [4]:
from deepcell.utils.data_utils import get_data




DATA_DIR2 = os.path.expanduser("/notebooks/aster_segmentation/datasets/")
filename2 = "Aster_fullsize_2D.npz"

train_dict, test_dict = deepcell.utils.get_data(os.path.join(DATA_DIR2, filename2))
X_train = train_dict['X'].astype("float64")
y_train = train_dict['y'].astype("float64")
X_test = test_dict['X'].astype("float64")
y_test = test_dict['y'].astype("float64")


#normalize each image to 0-256
for i in range(X_train.shape[0]):
    X_train[i] -= X_train[i].min()
    X_train[i] *= 256./X_train[i].max()
    
for i in range(X_test.shape[0]):
    X_test[i] -= X_test[i].min()
    X_test[i] *= 256./X_test[i].max()


#tile the dataset for bigger epochs
X_new = np.tile(X_train, (15,1,1,1))
y_new = np.tile(y_train, (15,1,1,1))

new_filename = '/notebooks/aster_segmentation/datasets/example_tiled_2D.npz'
np.savez(new_filename, X=X_new, y=y_new)
filename='example_tiled_2D.npz'

#print('X.shape: {}\ny.shape: {}'.format(X_train.shape, y_train.shape))

### Set up filepath constants

In [5]:
# the path to the data file is currently required for `train_model_()` functions
DATA_DIR = os.path.expanduser('/notebooks/aster_segmentation/datasets/')
filename='example_tiled_2D.npz'


# NOTE: Change DATA_DIR if you are not using `deepcell.datasets`
# DATA_DIR = os.path.expanduser(os.path.join('~', '.keras', 'datasets'))

DATA_FILE = os.path.join(DATA_DIR, filename)

# confirm the data file is available
assert os.path.isfile(DATA_FILE)

In [6]:
# Set up other required filepaths

# If the data file is in a subdirectory, mirror it in MODEL_DIR and LOG_DIR
PREFIX = os.path.relpath(os.path.dirname(DATA_FILE), DATA_DIR)

ROOT_DIR = '/notebooks/aster_segmentation/saved_networks'  # TODO: Change this! Usually a mounted volume
MODEL_DIR = os.path.abspath(os.path.join(ROOT_DIR, 'models', PREFIX))
LOG_DIR = os.path.abspath(os.path.join(ROOT_DIR, 'logs', PREFIX))

# create directories if they do not exist
for d in (MODEL_DIR, LOG_DIR):
    try:
        os.makedirs(d)
    except OSError as exc:  # Guard against race condition
        if exc.errno != errno.EEXIST:
            raise

### Set up training parameters

In [8]:
from tensorflow.keras.optimizers import SGD
from deepcell.utils.train_utils import rate_scheduler

fgbg_model_name = 'conv_model_std_61_test'

n_epoch = 1  # Number of training epochs
test_size = .10  # % of data saved as test
norm_method = "std"  # data normalization
receptive_field = 61  # should be adjusted for the scale of the data

optimizer = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

lr_sched = rate_scheduler(lr=0.01, decay=0.99)

# FC training settings
n_skips = 2  # number of skip-connections (only for FC training)
batch_size = 1  # FC training uses 1 image per batch

### First, create a foreground/background separation model

#### Instantiate the fgbg model

In [9]:
from deepcell import model_zoo

fgbg_model = model_zoo.bn_feature_net_skip_2D(
    n_features=3,
    receptive_field=receptive_field,
    n_skips=n_skips,
    n_conv_filters=32,
    n_dense_filters=128,
    norm_method = norm_method,
    input_shape=(None, None, 1),
    last_only=False)

#fgbg_model.load_weights("/notebooks/aster_segmentation/saved_networks/models/conv_model_std_61.h5") #load in a pretrained netowork

#### Train the model fgbg model

In [10]:
from deepcell.training import train_model_conv

fgbg_model = train_model_conv(
    model=fgbg_model,
    dataset=DATA_FILE,  # full path to npz file
    model_name=fgbg_model_name,
    test_size=test_size,
    optimizer=optimizer,
    n_epoch=n_epoch,
    batch_size=batch_size,
    transform=None,
    model_dir=MODEL_DIR,
    log_dir=LOG_DIR,
    lr_sched=lr_sched,
    rotation_range=180,
    flip=True,
    shear=False,
    zoom_range=(0.8, 1.2))

X_train shape: (4185, 240, 384, 1)
y_train shape: (4185, 240, 384, 1)
X_test shape: (465, 240, 384, 1)
y_test shape: (465, 240, 384, 1)
Output Shape: (None, None, None, 3)
Number of Classes: 3
Training on 2 GPUs
Epoch 1/1
2091/2092 [============================>.] - ETA: 0s - loss: 1.3981 - model_loss: 0.4712 - model_1_loss: 0.4579 - model_2_loss: 0.4403 - model_acc: 0.9190 - model_1_acc: 0.9182 - model_2_acc: 0.9209
Epoch 00001: val_loss improved from inf to 4.10375, saving model to /notebooks/aster_segmentation/saved_networks/models/conv_model_std_61_test.h5
2092/2092 [==============================] - 1056s 505ms/step - loss: 1.3977 - model_loss: 0.4711 - model_1_loss: 0.4578 - model_2_loss: 0.4402 - model_acc: 0.9191 - model_1_acc: 0.9182 - model_2_acc: 0.9209 - val_loss: 4.1037 - val_model_loss: 2.0813 - val_model_1_loss: 1.0401 - val_model_2_loss: 0.9525 - val_model_acc: 0.9558 - val_model_1_acc: 0.9562 - val_model_2_acc: 0.9572


### Run the model

#### Make predictions on test data

In [ ]:
test_images = fgbg_model.predict(X_test)[-1]

# np.savez( "/notebooks/docker_hole/deepcell-tf/output.npz", predicted = test_images, source = X_test, ground_truth =y_test)
# npzfile = np.load("/notebooks/docker_hole/deepcell-tf/output.npz")
# print(npzfile.files)

print('mask shape:', test_images.shape)

In [ ]:
import matplotlib.pyplot as plt

index = np.random.randint(low=0, high=X_test.shape[0])
#index = 24
print('Image number:', index)

fig, axes = plt.subplots(ncols=3, nrows=1, figsize=(15, 15), sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow(X_test[index, ..., 0], cmap="gray")
ax[0].set_title('Source Image')

ax[1].imshow(np.argmax(test_images[index], axis=-1))
ax[1].set_title('Prediction')

#ax[2].imshow(np.squeeze(test_dict['y'][index]))
ax[2].imshow(np.squeeze(y_test[index]))
ax[2].set_title('Ground Truth')

print(y_test[24][125][220])



In [ ]:
import cv2
import skimage.transform
from PIL import Image
import time

def check_val(name):
    num = ""
    for i in range(0,8):
        if name[i].isdigit():
            num += name[i]
    return int(num)

def file_sort(tif_arr):
    new_arr = [0]*len(tif_arr)
    for i in range(len(tif_arr)):
        new_arr[check_val(tif_arr[i])-1] = tif_arr[i]
    return new_arr



start = time.time()
#make sure to create an annotated_movies directory and download Zijie_Demo_ideo
base_dir= '/notebooks/aster_segmentation/Zijie_demo_Video/600um_length/'
file = "600_AR_0.75"
image_folder = os.path.join(base_dir,file)
video_name = os.path.join("/notebooks/aster_segmentation/annotated_movies/", file+"_annotated.avi" )



images = [img for img in os.listdir(image_folder) if img.endswith(".tif")]
#print(images)
images = file_sort(images)
#print(images)

image_arr = []
for i in range(len(images)):
    image_arr.append(np.array(Image.open(os.path.join(image_folder, images[i]))))
    

image_arr = np.array(image_arr)[...,np.newaxis].astype("float64")
#print(image_arr.min(axis = 0))
#image_arr = ((image_arr - image_arr.min(axis = 0)) /image_arr.max(axis = 0)) *65535

image_arr = skimage.transform.resize(image_arr, (image_arr.shape[0],240,384,1), anti_aliasing=False)

print(image_arr.max(), image_arr.min(), image_arr.shape)
image_arr = (image_arr/256).astype("float32")

for i in range(image_arr.shape[0]):
    image_arr[i] -= image_arr[i].min()
    image_arr[i] *= 256./image_arr[i].max()

predicted_images = fgbg_model.predict(image_arr)[-1]






frame = predicted_images[0]
height, width, layers = frame.shape

video = cv2.VideoWriter(video_name, 0, 10, (width,height))

for i in range(predicted_images.shape[0]):
    video.write(cv2.cvtColor(np.argmax(predicted_images[i], axis=-1).astype("uint8")*100,cv2.COLOR_GRAY2RGB))

cv2.destroyAllWindows()
video.release()
print(time.time()-start)






65535.0 1398.0 (179, 240, 384, 1)
